# 02 - Building the vector database

Ce notebook a pour objectif d’indexer les chunks extraits des thèses de l’École nationale des chartes dans des bases vectorielles, en suivant les configurations spécifiées dans le fichier `config.yml`.

Dans le cadre d'une architecture RAG, cette étape correspond à la mise en place du composant Retriever.

> Dans le notebook précédent, nous avons préparé les chunks selon une stratégie adaptée à notre cas d’usage et stocké dans un fichier csv.

Une fois les chunks encodés (embedded), ils sont stockés dans une base de données vectorielle. Lorsqu’un utilisateur saisit une requête, celle-ci est à son tour encodée par le même modèle, puis comparée aux vecteurs présents dans la base pour identifier les documents les plus similaires.

Le défi technique principal est donc le suivant :

> Étant donné un vecteur de requête, retrouver rapidement ses **k plus proches voisins** dans la base vectorielle, c’est-à-dire les k documents les plus pertinents.

In [1]:
## Imports globaux
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from utils.io_utils import read_yaml
from utils.vector_store import VectorDB
import umap.umap_ as umap
import plotly.express as px
import joblib
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Chargement de la configuration
config = read_yaml("../config.yml")
data_path = "../data/raw/encpos_chunked_tok_512_51.csv"
defaults = config.get("defaults", {})

In [3]:
## Chargement des données à indexer
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Fichier introuvable : {data_path}. Lancez d'abord le notebook de chunking.")
df = pd.read_csv(data_path, sep="\t")
print(f"Nombre de chunks à indexer : {len(df)}")

Nombre de chunks à indexer : 39377


On prepare les configurations d'indexation vectorielle à partir du fichier de configuration `config.yml`.

In [4]:
## Génération des configurations d'indexation
vector_indexing = []
for entry in config.get("vector_indexing", []):
    model_id = entry["model_id"]
    model = next((m for m in config["embedding_models"] if m["id"] == model_id), None)
    if not model:
        raise ValueError(f"Modèle non trouvé : {model_id}")

    for backend in entry["backends"]:
        suffix = f"{model_id}_{backend}"
        name = f"{model['name']} - {backend.upper()}"
        collection_name = f"{defaults.get('collection_prefix', 'encpos')}_{model_id}"
        path = os.path.join(defaults.get("base_path", "data/vectordb"), suffix)

        vector_indexing.append({
            "name": name,
            "backend": backend,
            "embedding_model": model["model_path"],
            "metric": defaults.get("metric", "cosine"),
            "text_column": defaults.get("text_column", "full_chunk"),
            "metadata_columns": defaults.get("metadata_columns", []),
            "path": path,
            "qdrant_collection_name": collection_name,
            "k": defaults.get("k", 10),
            "force_rebuild": defaults.get("force_rebuild", False)
        })


Pour indexer efficacement nos données dans une base vectorielle contenant potentiellement des milliers de documents, il est nécessaire de choisir deux éléments clés :

- Une **métrique de distance** pour comparer les vecteurs (ex. similarité cosinus, distance euclidienne etc.) ;

- Un **algorithme de recherche des plus proches voisins** (nearest neighbors search) pour retrouver rapidement les documents pertinents.

Dans la cellule suivante nous avons mis en place une boucle principale qui construit une base vectorielle pour chaque configuration définie dans le fichier `config.ym`l.
L’objectif est de tester différentes combinaisons de modèles d’embeddings et de backends de stockage.

Les deux backends actuellement pris en charge sont :

- Faiss : très rapide pour l’indexation et la recherche, mais les filtres sur les métadonnées sont limités ;

- LanceDB : plus lent à l’indexation, mais permet des requêtes complexes sur les métadonnées par exemple en SQL.

Nous avons choisi d’utiliser la similarité cosinus comme métrique de comparaison entre les vecteurs. Elle mesure l’angle entre deux vecteurs, ce qui permet de comparer leur direction indépendamment de leur norme. Cela nécessite de normaliser tous les vecteurs (c’est-à-dire de leur donner une norme unitaire) avant l’indexation ou la recherche.

Pour faciliter l’indexation, nous avons développé une abstraction Python appelée `VectorDB` qui prend en charge :

- La normalisation des vecteurs

- La création des bases vectorielles et leur persistance

- L’indexation des embeddings

- La recherche

Cette abstraction nous permet de comparer différents modèles et backends de manière uniforme, et de les évaluer dans des conditions équitables.



In [5]:
%%time
## Boucle principale d’indexation
# 2h55 pour indexer les chunks dans 6 vectorstore
# UMAP :
UMAP_PROJECTION = False
#df = df.sample(n=50)

def get_stored_vectors(db: VectorDB) -> np.ndarray:
    if db.backend == "faiss":
        # db.index.index est un IndexFlat ou autre structure FAISS
        return np.array(db.index.index.reconstruct_n(0, db.index.index.ntotal))

    elif db.backend == "lancedb":
        df = db.index.to_pandas()
        return np.vstack(df["vector"].values)  # Chaque vecteur est une liste → stack en matrice

    else:
        raise ValueError(f"Backend {db.backend} non supporté")

for conf in vector_indexing:
    print("\n--- Indexation en cours ---")
    print("Nom:", conf["name"])

    db = VectorDB(
        backend=conf["backend"],
        embedding_model=conf["embedding_model"],
        metric=conf["metric"],
        path=conf["path"],
        k=conf["k"],
        force_rebuild=conf["force_rebuild"]
    )


    db.add_from_dataframe(
        df=df,
        text_column=conf["text_column"],
        metadata_columns=conf["metadata_columns"]
    )

    db.save()
    print("📦 Indexation terminée pour:", conf["name"])

    if UMAP_PROJECTION:
        # Génération de la projection UMAP
        print("→ Génération de la projection UMAP pour:", conf["name"])
        umap_path = os.path.join("scripts", conf["path"], "projection_umap.npy")
        labels_path = os.path.join("scripts", conf["path"], "projection_labels.pkl")
        if os.path.isfile(umap_path) and os.path.isfile(labels_path):
            print("🔁 Projection UMAP déjà générée, chargement depuis le disque.")
        else:
            print("🛠️ Calcul de la projection UMAP...")
            vectors = get_stored_vectors(db)
            reducer = umap.UMAP(n_components=2, random_state=42, n_jobs=-1)
            reduced = reducer.fit_transform(np.array(vectors))
            joblib.dump(reducer, os.path.join(conf["path"], "umap_model.joblib"))
            np.save(umap_path, reduced)
            with open(labels_path, "wb") as f:
                pickle.dump(
                    df[["author", "position_name", "year"]].to_dict("records"), f
                )
            print("✅ Projection UMAP sauvegardée dans:", umap_path)


--- Indexation en cours ---
Nom: CamemBERT Large - LANCEDB
📦 Initialisation de LanceDB à data/vectordb/camembert-large_lancedb
🆕 Table LanceDB 'camembert-large_lancedb' à créer lors de l'indexation.


Préparation des documents pour lancedb: 100%|██████████| 39377/39377 [00:01<00:00, 21462.39it/s]


🧠 Création de la table LanceDB 'camembert-large_lancedb' à partir des documents...
📦 Indexation terminée pour: CamemBERT Large - LANCEDB

--- Indexation en cours ---
Nom: CamemBERT Base - LANCEDB
📦 Initialisation de LanceDB à data/vectordb/camembert-base_lancedb
🆕 Table LanceDB 'camembert-base_lancedb' à créer lors de l'indexation.


Préparation des documents pour lancedb: 100%|██████████| 39377/39377 [00:01<00:00, 37788.02it/s]


🧠 Création de la table LanceDB 'camembert-base_lancedb' à partir des documents...
📦 Indexation terminée pour: CamemBERT Base - LANCEDB

--- Indexation en cours ---
Nom: Multilingual DistilUSE - LANCEDB
📦 Initialisation de LanceDB à data/vectordb/multilingual_lancedb
🆕 Table LanceDB 'multilingual_lancedb' à créer lors de l'indexation.


Préparation des documents pour lancedb: 100%|██████████| 39377/39377 [00:01<00:00, 36957.65it/s]


🧠 Création de la table LanceDB 'multilingual_lancedb' à partir des documents...
📦 Indexation terminée pour: Multilingual DistilUSE - LANCEDB
CPU times: user 4min 24s, sys: 3min 8s, total: 7min 32s
Wall time: 52min 2s


## Visualisation des résultats

### Carte UMAP des embeddings

Durant la boucle d’indexation, nous avons également généré un modèle de projection des embeddings pour chaque configuration. Cette projection permet de visualiser la distribution des chunks dans un espace 2D via un algorithme de réduction de dimensionnalité des vecteurs d'embeddings.

Nous utilisons ici UMAP (Uniform Manifold Approximation and Projection) mais il en existe d'autres comme PCA, t-SNE, PaCMAP etc.

Sur le graphique ci-dessous, on observe une représentation spatiale des documents de la base de connaissances.
Chaque point correspond à un document, représenté par son embedding vectoriel, qui encode son contenu sémantique.

L'intution ici est que plus deux documents sont proches en termes de sens, plus leurs vecteurs devraient l’être également dans l’espace vectoriel.

L’embedding de la requête utilisateur est également représenté sur le graphique.

L’objectif ici est de retrouver les k documents dont le sens est le plus proche de cette requête.

Les projections nous aident à visualiser la distribution des documents dans l’espace vectoriel et à identifier les éventuels clusters ou regroupements de chun similaires

In [ ]:
## Exemple d’utilisation et visualisation
example_conf = vector_indexing[0]
print(
    f"\n🔍 Configuration d'exemple : {example_conf['name']}\n"
)

In [ ]:
if example_conf:
        print(f"\n🎯 Exemple : utilisation du retriever {example_conf['name']}")

        db = VectorDB(
            backend=example_conf["backend"],
            embedding_model=example_conf["embedding_model"],
            metric=example_conf["metric"],
            path=example_conf["path"],
            k=5,
            force_rebuild=False
        )

        query = "Quel est le temps de Léon Marchand au 400m papillon aux Jeux Olympiques de Paris de 2025 ?"
        results = db.query(query)

        print("\n🧠 Résultats de la requête :")
        for i, r in enumerate(results[:5]):
            print(i, r)

        print(f"\n🎨 Affichage de la projection UMAP :{example_conf['name']}")
        umap_path = os.path.join(example_conf["path"], "projection_umap.npy")
        labels_path = os.path.join(example_conf["path"], "projection_labels.pkl")
        # load previously fitted UMAP reducer
        reducer = joblib.load(os.path.join(example_conf["path"], "umap_model.joblib"))
        if os.path.exists(umap_path) and os.path.exists(labels_path):
            emb_2d = np.load(umap_path)
            with open(labels_path, "rb") as f:
                labels = pickle.load(f)

            # Requête utilisateur (embedding + projection)
            user_embedding = db.embed_query(query)


            query_proj = reducer.transform(reducer.transform(np.array(user_embedding).reshape(1, -1)))

            # Création du DataFrame pour Plotly
            df_plot = pd.DataFrame.from_dict(
                [
                    {
                        "x": emb_2d[i, 0],
                        "y": emb_2d[i, 1],
                        "source": f"{labels[i].get('author', 'Inconnu')}",
                        "extract": f"{labels[i].get('position_name', '')} : {labels[i].get('text', '')[:100]}...",
                        "symbol": "circle",
                        "size_col": 4,
                    }
                    for i in range(len(labels))
                ]
                + [
                    {
                        "x": query_proj[0, 0],
                        "y": query_proj[0, 1],
                        "source": "User query",
                        "extract": query,
                        "symbol": "star",
                        "size_col": 100,
                    }
                ]
            )

            # Visualisation avec Plotly
            fig = px.scatter(
                df_plot,
                x="x",
                y="y",
                color="source",
                hover_data=["extract"],
                size="size_col",
                symbol="symbol",
                color_discrete_map={"User query": "black"},
                width=1000,
                height=700,
            )
            fig.update_traces(
                marker=dict(opacity=1, line=dict(width=0, color="DarkSlateGrey")),
                selector=dict(mode="markers"),
            )
            fig.update_layout(
                legend_title_text="<b>Chunk source</b>",
                title=f"<b>2D Projection of Positions de thèses embeddings via UMAP ({example_conf['embedding_model']})</b>",
                plot_bgcolor="white",
            )
            fig.show()
        else:
            print("⚠️ Fichiers de projection manquants. Relancer l’étape UMAP si besoin.")


➡️ Notebook suivant : [03-assemble_rag.ipynb](./03-assemble_rag.ipynb)